In [5]:
from dateutil.parser import parse 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet as pr
#import mplcursors
from datetime import datetime

Matplotlib created a temporary config/cache directory at C:\Users\MATHEU~1\AppData\Local\Temp\matplotlib-bjadgdz8 because the default path (C:\Users\Matheus Alves\.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Importing plotly failed. Interactive plots will not work.


In [6]:
import warnings
warnings.filterwarnings("ignore")

# Carregando Base de Dados, Dividindo entre Teste e Treino e Alterando para Padrão Prophet


In [303]:
rainfall = pd.read_csv(r"C:\Users\Matheus Alves\Documents\git_tcc\time_series_precipitation\Dataset\dados_chuvas1.csv", parse_dates=True)

In [304]:
rainfall['Date'] = pd.to_datetime(rainfall['Date'], format="%Y/%m/%d")
#rainfall['Date'] = rainfall['Date'].dt.strftime('%Y-%m')

In [305]:
train = rainfall.iloc[:-14]
test = rainfall.iloc[-14:]

In [306]:
 train.columns = ['ds','y']

# Instanciando Prophet e Fazendo uma Previsão Básica

In [320]:
rainfall['month'] = pd.DatetimeIndex(rainfall.Date).month
rainfall_summer = rainfall[rainfall.month == 1]

In [ ]:
#adicionando os meses de verão para uma previsão mais precisa
summer = pd.DataFrame({'holiday': 'summer',
                       'ds': pd.to_datetime(['2020-12-31'	
	'2021-01-31'	
	'2021-02-28'	
	'2021-03-31'])}

In [307]:
model = pr(interval_width=0.95, daily_seasonality=False)
model.fit(train)
future = model.make_future_dataframe(periods=14, freq='M')

21:10:23 - cmdstanpy - INFO - Chain [1] start processing
21:10:23 - cmdstanpy - INFO - Chain [1] done processing


In [308]:
rainfall_pred = model.predict(future)

In [309]:
rainfall_pred

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1984-01-31,130.179961,150.307041,421.591837,130.179961,130.179961,156.943391,156.943391,156.943391,156.943391,156.943391,156.943391,0.0,0.0,0.0,287.123352
1,1984-02-29,130.198527,112.625874,393.952380,130.198527,130.198527,115.733091,115.733091,115.733091,115.733091,115.733091,115.733091,0.0,0.0,0.0,245.931618
2,1984-03-31,130.218373,59.513690,328.918191,130.218373,130.218373,68.706070,68.706070,68.706070,68.706070,68.706070,68.706070,0.0,0.0,0.0,198.924443
3,1984-04-30,130.237578,-53.411577,214.243707,130.237578,130.237578,-46.807975,-46.807975,-46.807975,-46.807975,-46.807975,-46.807975,0.0,0.0,0.0,83.429603
4,1984-05-31,130.257424,-52.038826,215.289658,130.257424,130.257424,-54.562856,-54.562856,-54.562856,-54.562856,-54.562856,-54.562856,0.0,0.0,0.0,75.694568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,2021-09-30,138.917221,-39.695621,215.313471,138.916919,138.917513,-53.323495,-53.323495,-53.323495,-53.323495,-53.323495,-53.323495,0.0,0.0,0.0,85.593726
453,2021-10-31,138.936524,-13.403506,255.849661,138.936195,138.936876,-15.255387,-15.255387,-15.255387,-15.255387,-15.255387,-15.255387,0.0,0.0,0.0,123.681137
454,2021-11-30,138.955204,10.907234,273.160249,138.954836,138.955593,1.752539,1.752539,1.752539,1.752539,1.752539,1.752539,0.0,0.0,0.0,140.707744
455,2021-12-31,138.974507,97.836310,358.417285,138.974073,138.974941,91.962892,91.962892,91.962892,91.962892,91.962892,91.962892,0.0,0.0,0.0,230.937399


In [310]:
previsao = rainfall_pred[['ds','yhat']].iloc[-14:]

In [311]:
test

,Date,Precipitation
443,2020-12-31,273.2
444,2021-01-31,215.5
445,2021-02-28,180.6
446,2021-03-31,138.4
447,2021-04-30,54.4
448,2021-05-31,44.4
449,2021-06-30,16.2
450,2021-07-31,39.4
451,2021-08-31,44.4
452,2021-09-30,39.5


In [312]:
previsao.columns = ['Date','Precipitation']
previsao.round(1)

,Date,Precipitation
443,2020-12-31,234.6
444,2021-01-31,299.0
445,2021-02-28,254.4
446,2021-03-31,215.8
447,2021-04-30,91.1
448,2021-05-31,80.9
449,2021-06-30,66.4
450,2021-07-31,49.4
451,2021-08-31,40.4
452,2021-09-30,85.6


# Avaliando o Modelo

In [313]:
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError

In [314]:
real_value = np.array(test['Precipitation'])
pred_value = np.array(previsao['Precipitation'])

In [315]:
smape = MeanAbsolutePercentageError(symmetric=True)
smape(real_value, pred_value)

0.4193089426735173